**Import and Initial Inspection**

The dataset utilized in this project is derived from my Independent Research Project, featuring data sourced from the Canadian Department of Fisheries, the Bedford Institute of Oceanography, and GEBCO.The data represents a selected subset of variables of that will be used in the final project. 

This data has presence and absence locations for a species of Glass Sponge that has gloablly unique aggregations on the Nova Scotian Shelf. As deep sea surveys are few and far inbetween, modelling the potential habitat of the Glass Sponge is required in order to support responsible marine spatial planning and conservation efforts for Nova Scotias deep sea sponge ecosystems.

The target variable 'Presence', encoded as a binary classification where Presence=1 signifies observed presence and Absence=2 indicates an observed absence. The explanatory variables comprise a diverse array of oceanogprahic factors intended to deliniate potential prefered habitats.

This notebook serves as a guide to binary classification using a deep learning model called LSTM (Long Short-Term Memory) RNN networks. It shows initial data preprocessing steps such as column removal and data normalization, partitioning the dataset into training and testing, training the LSTM model, and evaluation of the model's ability to predict presence or absence based on oceanographic factors.

In [1]:
import pandas as pd

data = pd.read_csv(r'C:\NSCC\winter\DataMiningModelling\DeepLearning\GlassSponge_PA.csv')

data.head()

data.columns

data= data.drop(['MISSION', 'YEAR', 'comm','OID_', 'SOURCE'], axis=1)

data.rename(columns={
    'Var_Slope': 'Slope',
    'Var_Bathy': 'Depth',
    'Var_Current_Bottom_Mean': 'Bottom Current Mean',
    'Var_Current_Surface_Mean': 'Surface Current Mean',
    'Var_pp_mean': 'Primary Productivity Mean',
    'Var_Salinity_Bottom_Mean': 'Bottom Salinity Mean',
    'Var_Salinity_Surface_Mean': 'Surface Salinity Mean',
    'Var_Mean_Bottom_Temp': 'Bottom Temp Mean',
    'Var_Mean_Surface_Temp': 'Surface Temp Mean',
    'Var_Max_Bottom_Temp' : 'Bottom Temp Max',
    'Var_Min_Bottom_Temp' : 'Bottom Temp Min',
    'Var_Min_Surface_Temp': 'Surface Temp Min',
    'Var_Max_Surface_Temp': 'Surface Temp Max', }, inplace=True)

data.columns

Index(['Latitude', 'Longitude', 'Presence', 'Slope', 'Depth',
       'Bottom Current Mean', 'Surface Current Mean',
       'Primary Productivity Mean', 'Bottom Salinity Mean',
       'Surface Salinity Mean', 'Bottom Temp Mean', 'Bottom Temp Max',
       'Bottom Temp Min', 'Surface Temp Mean', 'Surface Temp Min',
       'Surface Temp Max'],
      dtype='object')

**Data Cleaning**

All rows with Null values are removed

In [ ]:
null_values = data.isnull().sum()

print("Number of null values in each column:\n", null_values)

data = data.dropna()

print("New DataFrame shape after dropping rows with null values:", data.shape)

**Exploring distributions of variables**

First, I wanted to explore whether there were differences in the distribution of variables for my binary target, categorizing the data into 'Absence' and 'Presence' groups. This initial step was done to visualize how the variables' distribution differs across the two categories. 

I also plotted the overall distribution of each variable. This overview was aimed at determining the necessity for data preprocessing steps such as trimming outliers or applying transformations to improve the shape of the distributions. This highlighted lots of outliers and non-normal distributions that will be trimmed in later data preparation steps in order to optimize the analysis.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


for column in data.drop(['Presence', 'Latitude', 'Longitude'], axis=1).columns:
    # Create a figure with 2 subplots side by side
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20, 6)) 

    # Plot variables densities by presence column
    sns.kdeplot(data[data['Presence'] == 0][column], label='Absent', fill=True, ax=axs[0])
    sns.kdeplot(data[data['Presence'] == 1][column], label='Present', fill=True, ax=axs[0])
    axs[0].set_title(f'Distribution of {column} by Presence')
    axs[0].set_xlabel(column)
    axs[0].set_ylabel('Density')
    axs[0].legend()

    # Plot overall density of variables
    sns.kdeplot(data[column], label='Overall', fill=True, ax=axs[1])
    axs[1].set_title(f'Overall Distribution of {column}')
    axs[1].set_xlabel(column)
    axs[1].set_ylabel('Density')
    axs[1].legend()

    plt.show()


**Data Preprocessing**

Here I did preprocessing through outlier trimming, scaling, and transforming specific variables. It starts by removing outliers for the predictor values that lie beyond two standard deviations from the mean. Then the predictors are scaled using the RobustScaler, which reduces the influence of outliers. For negatively skewed variables, a Box-Cox transformation is applied to make their distribution more symmetric. Transformations were attempted on all variables, although only kept for the variables where the transfromation impoved the overal distribution. Finally, the code visualizes the original, scaled, and transformed distributions for each variable.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import boxcox


np.random.seed(2010)

# Standard deviation trimming
num_std_dev = 2
mean = data.drop(columns=['Presence', 'Latitude', 'Longitude']).mean()
std_dev = data.drop(columns=['Presence', 'Latitude', 'Longitude']).std()
outliers = ((data.drop(columns=['Presence', 'Latitude', 'Longitude']) - mean).abs() > num_std_dev * std_dev).any(axis=1)
trimmed_data = data[~outliers]

# After trimming, reset the index 
trimmed_data.reset_index(drop=True, inplace=True)

# Scale the Predictors with robust scaler
predictors = trimmed_data.drop(columns=['Presence', 'Latitude', 'Longitude'])
target = trimmed_data['Presence']
scaler = RobustScaler()
scaled_predictors = scaler.fit_transform(predictors)
scaled_data = pd.DataFrame(scaled_predictors, columns=predictors.columns)
scaled_data['Presence'] = target

df_transformed = scaled_data[['Presence','Surface Current Mean','Slope', 'Depth', 'Bottom Current Mean', 
                              'Primary Productivity Mean', 'Surface Salinity Mean', 'Surface Temp Mean', 
                              'Surface Temp Min', 'Surface Temp Max']].copy()

negatively_skewed_variables = [ 'Bottom Salinity Mean', 'Bottom Temp Mean', 'Bottom Temp Max', 'Bottom Temp Min']

# Apply Box-Cox transformation to negatively skewed variables
for variable in negatively_skewed_variables:
    data_positive = scaled_data[variable] - scaled_data[variable].min() + 1
    transformed_data, _ = boxcox(data_positive)
    df_transformed[variable] = transformed_data

# Plot Distributions
for column in df_transformed.drop(columns=['Presence']).columns:
    if column in negatively_skewed_variables:
        fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(20, 6)) 
    else:
        fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(13, 6)) 

    # Original Distribution
    sns.kdeplot(data[column], label='Original', fill=True, ax=axs[0])
    axs[0].set_title(f'Original Distribution of {column}')
    axs[0].set_xlabel(column)
    axs[0].set_ylabel('Density')
    axs[0].legend()

    # Scaled Distribution
    sns.kdeplot(scaled_data[column], label='Scaled', fill=True, ax=axs[1])
    axs[1].set_title(f'Trimmed and Scaled Distribution of {column}')
    axs[1].set_xlabel(column)
    axs[1].set_ylabel('Density')
    axs[1].legend()

    # Transformed Distribution (only for variables that were transformed)
    if column in negatively_skewed_variables:
        sns.kdeplot(df_transformed[column], label='Transformed', fill=True, ax=axs[2])
        axs[2].set_title(f'Transformed Distribution of {column}')
        axs[2].set_xlabel(column)
        axs[2].set_ylabel('Density')
        axs[2].legend()

    plt.tight_layout()
    plt.show()



**Converting to Tensors**

This code selects predictors and the target variable from the dataset, excluding 'Presence' column from the predictors. Then predictors and target variables are converted into PyTorch tensors.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision import transforms
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Select predictors and target
predictors = df_transformed.drop(['Presence'], axis=1).values
target = df_transformed['Presence'].values

# Convert to PyTorch tensors
X = torch.Tensor(predictors).unsqueeze(1) 
y = torch.Tensor(target)

# Create a dataset
dataset = TensorDataset(X, y)

**Splitting Testing and Training**

In this section of code the dataset is divided into training and testing with an 80-20 split, ensuring a balanced representation of both target classes.

 Originally, the code didn't have the `stratify=y` parameter, leading to a skewed distribution of the binary target y in the splits. This imbalance skewed the model towards predominantly predicting one class, resulting in a biased model. By introducing `stratify=y`, the distribution of the binary outcomes presence (1) and absence (0) is even across both training and testing sets. This adjustment significantly improved the models ability to generalize and accurately predict both classes. This little addition of code is the key for this models performance. Various runs where attempted before adding this parameter and all of them performed incredibly poorly with AOC values indicating 'worse than random'. None of the other preprocessing steps or additions to the code showed such a stark improvement as this one. Initially I didn't think including this was needed becuase the dataset was already evenly split between (1) and (0), although I learned that wasn't enough. 

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2010)


**Define Model**

Defines a PyTorch model class for binary classification using an LSTM architecture. 

In [ ]:
class BinaryClassificationLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1):
        super(BinaryClassificationLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        lstm_out, (hn, _) = self.lstm(x)
        lstm_out = lstm_out.contiguous().view(batch_size * seq_len, -1)
        out = self.fc(lstm_out)
        out = out.view(batch_size, seq_len, -1).squeeze(-1)
        out = out[:, -1]
        return out


**Parameters**

This code first sets the device to CUDA if available, otherwise defaults to CPU. Then initializes the LSTM model with defined input size, hidden layer size, and output size. It also defines Binary Cross Entropy with Logits as the loss function and Adam as the optimizer.

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = predictors.shape[1] 
output_size = 1
hidden_size = 64  
model = BinaryClassificationLSTM(input_size, hidden_size).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


**Training Model**

This code trains the LSTM model for 100 epochs and calculates accuracy,loss, precison and recall for each epoch to monitor training progress.


Using Root Mean Square Error (RMSE) for binary classification is possible but not ideal because RMSE focuses on the magnitude of prediction errors, which is more relevant to continuous data. In binary classification, where outcomes are categorical (0 or 1), RMSE doesn't reflect the model's classification accuracy or its ability to distinguish between the classes. Metrics like accuracy, precision, recall, F1, AUC-ROC and confusion matrix are more appropriate as they evaluate classification performance based on correct and incorrect predictions.

In [ ]:
import torch
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)  
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    
    model.eval()
    with torch.no_grad():
        # Calculate probabilities and binary predictions
        train_outputs_probs = torch.sigmoid(model(X_train))
        predictions = (train_outputs_probs >= 0.5).float()
        
        # Convert predictions and y_train to CPU numpy arrays necessary for sklearn metrics
        predictions_np = predictions.cpu().numpy()
        y_train_np = y_train.cpu().numpy()
        
        # Calculate accuracy, precision, recall, and F1 score
        accuracy = accuracy_score(y_train_np, predictions_np)
        precision = precision_score(y_train_np, predictions_np, zero_division=0)
        recall = recall_score(y_train_np, predictions_np, zero_division=0)
        f1 = f1_score(y_train_np, predictions_np, zero_division=0)
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, '
          f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')





**Run Model on Testing Data**

1. *Accuracy* measures the proportion of true results (both true positives and true negatives) among the total number of predictions. An accuracy of 94.80% means that the model's predictions are correct 95.8% of the time.

2. *Precision* measures the proportion of true positive outcomes in the predictions made by your model. A precision of 94.07% means that the model predicted the positive class correctly 94.07% of the time. The results show a high precision, which indicates a low false positive rate.

3. *Test Recall (Sensitivity)* measures the proportion of actual positives that were correctly identified by the model. A recall of 98.33% indicates that the model successfully identified nearly all true positives.

4. *F1 Scores*  is the harmonic mean of precision and recall, providing a single metric to assess balance between them. An F1 score of 96.15% shows that the model achieves a balence between precision and recall. This is particularly useful when you care equally about false positives and false negatives, which in this case we do.

*Overall Assessment of Model*

This model demonstrates excellent performance across all the metrics (Accuracy, Precision, Recall, F1, AUC-ROC, Confusion Matrix). It has a high degree of accuracy in distinguishing between presence and absence, with a particularly high ability to identify true positives (high recall) and maintaining a low rate of false positives (high precision). The high AUC-ROC score indicates that the model's probability scores for the positive class was also good. 


It's suprising that this model, which is a type of Recurrent Neural Network (RNN) achieved such sucess at predicting this datasets binary predictions as this type of deep learning model is typically associated with time series analysis. This shows that RNNs are adeptive and can handle binary predictions and extract relevant patterns for a wide range of tasks, not limited to their conventional applications.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch


with torch.no_grad():
    model.eval()  
    test_outputs = torch.sigmoid(model(X_test))
    
    # Convert to binary predictions
    predictions = (test_outputs >= 0.5).float()
    
    # Convert tensors to numpy arrays for use with sklearn metrics
    predictions_np = predictions.cpu().numpy()
    y_test_np = y_test.cpu().numpy()
    
    # Calculate accuracy, precision, recall, and F1 score
    accuracy = accuracy_score(y_test_np, predictions_np)
    precision = precision_score(y_test_np, predictions_np, zero_division=0)
    recall = recall_score(y_test_np, predictions_np, zero_division=0)
    f1 = f1_score(y_test_np, predictions_np, zero_division=0)
    
    # Print all metrics
    print(f'Test Accuracy: {accuracy:.4f}')
    print(f'Test Precision: {precision:.4f}')
    print(f'Test Recall: {recall:.4f}')
    print(f'Test F1 Score: {f1:.4f}')



**Model Evaluation via AUC-ROC**

*Area Under the Receiver Operating Characteristic (ROC) curve* is a performance measurement for classification problems such as binary predictions seen in this model. An AUC-ROC of 99.03% indicates the model is excellent at distinguishing between the positive and negative classes.

In [ ]:
from sklearn.metrics import roc_auc_score
import torch

model.eval()

with torch.no_grad():
  
    logits = model(X_test)
    probabilities = torch.sigmoid(logits)
    probabilities_np = probabilities.cpu().numpy()
    y_test_np = y_test.cpu().numpy()
    auc_roc = roc_auc_score(y_test_np, probabilities_np)
    
    print(f"AUC-ROC: {auc_roc:.4f}")





**Model Evaluation via Confusion Matrix**

The confusion matrix gives breaks down the performance:
True Negatives: 379
False Positives: 52
False Negatives: 14
True Positives: 825
This shows that the model has relatively few false positives and false negatives. It predicted postive incorrecrly 52 times and negative incorrectly 14 times. 

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = (probabilities >= 0.5).float()  # Convert probabilities to binary predictions
conf_matrix = confusion_matrix(y_test.cpu().numpy(), y_pred.cpu().numpy())

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', cbar=False)

plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.xticks(ticks=[0.5, 1.5], labels=['0', '1'])  
plt.yticks(ticks=[0.5, 1.5], labels=['0', '1'], rotation=0) 

plt.show()
